<a href="https://colab.research.google.com/github/krkr5628/Crypto_Analysis_Colab/blob/main/1_Data_Feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas_ta

In [ ]:
pip install ta

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#
from tqdm import tqdm
import ta
import pandas_ta as pata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

######################

**데이터 호출**
######################

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL60_INDICATOR3'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_Data_30m.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_Data_1m_Micro_Indicator3.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
# 데이터프레임의 길이 계산
total_length = len(data)

# 4등분으로 나눈 길이 계산
split_length = total_length // 4

# 두번째 부분 데이터 분리
data = data.iloc[split_length*2:split_length*3]
data.head()

In [ ]:
#SAVE
file_path2 = '/content/drive/MyDrive/Data/SOL_Data_Micro.csv'
data.to_csv(file_path2, index=False);

######################

**테스트 데이터 분리**
######################

In [ ]:
# 데이터프레임의 길이 계산
total_length = len(data)

# 4등분으로 나눈 길이 계산
split_length = total_length // 4

# 두번째 부분 데이터 분리
data = data.iloc[split_length*2:split_length*3]
data.head()

In [ ]:
#SAVE
file_path2 = '/content/drive/MyDrive/Data/SOL_Data_Test.csv'
data.to_csv(path2, index=False);

######################

**데이터 지표 추가**
######################

In [ ]:
# open_time 열을 datetime 형식으로 변환
data['open_time'] = pd.to_datetime(data['open_time'])

# 시각과 분만 추출하여 time 열 생성
data['time'] = data['open_time'].dt.strftime('%H:%M')

# ATR 계산 함수
def calculate_atr(df, periods):
    for period in periods:
        df[f'atr_{period}'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=period)
    return df

# VWAP 계산 함수
def calculate_vwap(df):
    df['vwap'] = ta.volume.volume_weighted_average_price(df['high'], df['low'], df['close'], df['volume'])
    return df

# Stochastic Oscillator 계산 함수
def calculate_stoch(df, periods):
    for period, smooth in periods:
        df[f'stoch_%k_{period}_{smooth}'] = ta.momentum.stoch(df['high'], df['low'], df['close'], window=period, smooth_window=smooth)
        df[f'stoch_%d_{period}_{smooth}'] = ta.momentum.stoch_signal(df['high'], df['low'], df['close'], window=period, smooth_window=smooth)
    return df

# OBV 계산 함수
def calculate_obv(df):
    df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    return df

# Bollinger Bands 계산 함수
def calculate_bollinger_bands(df, periods):
    for period in periods:
        bollinger = ta.volatility.BollingerBands(df['close'], window=period)
        df[f'bollinger_hband_{period}'] = bollinger.bollinger_hband()
        df[f'bollinger_lband_{period}'] = bollinger.bollinger_lband()
    return df

# Ichimoku 계산 함수
def calculate_ichimoku(df, periods):
    for period in periods:
        df[f'ichimoku_base_{period}'] = ta.trend.ichimoku_base_line(df['high'], df['low'], window1=period)
        df[f'ichimoku_conversion_{period}'] = ta.trend.ichimoku_conversion_line(df['high'], df['low'], window1=period)
    return df

# Supertrend 계산 함수
def calculate_supertrend(df, settings):
    df = df.copy()
    for period, multiplier, atr_period in settings:
        hl2 = (df['high'] + df['low']) / 2
        df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=atr_period)
        df['upperband'] = hl2 + (multiplier * df['atr'])
        df['lowerband'] = hl2 - (multiplier * df['atr'])
        df['in_uptrend'] = True

        for current in tqdm(range(1, len(df.index)), desc=f'Calculating Supertrend {period}-{multiplier}-{atr_period}', leave=False):
            previous = current - 1

            if df['close'].iloc[current] > df['upperband'].iloc[previous]:
                df.loc[df.index[current], 'in_uptrend'] = True
            elif df['close'].iloc[current] < df['lowerband'].iloc[previous]:
                df.loc[df.index[current], 'in_uptrend'] = False
            else:
                df.loc[df.index[current], 'in_uptrend'] = df['in_uptrend'].iloc[previous]

                if df['in_uptrend'].iloc[current] and df['lowerband'].iloc[current] < df['lowerband'].iloc[previous]:
                    df.loc[df.index[current], 'lowerband'] = df['lowerband'].iloc[previous]

                if not df['in_uptrend'].iloc[current] and df['upperband'].iloc[current] > df['upperband'].iloc[previous]:
                    df.loc[df.index[current], 'upperband'] = df['upperband'].iloc[previous]

        df[f'supertrend_upper_{period}_{multiplier}_{atr_period}'] = df['upperband']
        df[f'supertrend_lower_{period}_{multiplier}_{atr_period}'] = df['lowerband']
        df[f'supertrend_in_uptrend_{period}_{multiplier}_{atr_period}'] = df['in_uptrend']
    return df

# 모든 계산 수행
atr_periods = [5, 10, 14, 20, 50]
stoch_periods = [(14, 3), (21, 5), (9, 3), (5, 2), (20, 7)]
bollinger_periods = [10, 20, 50, 100, 200]
ichimoku_periods = [9, 26, 52, 100, 200]
supertrend_settings = [(7, 3, 14), (10, 3, 20), (14, 2, 10), (20, 4, 50), (50, 5, 5)]

data = calculate_atr(data, atr_periods)
data = calculate_vwap(data)
data = calculate_stoch(data, stoch_periods)
data = calculate_obv(data)
data = calculate_bollinger_bands(data, bollinger_periods)
data = calculate_ichimoku(data, ichimoku_periods)
data = calculate_supertrend(data, supertrend_settings)

In [ ]:
# 지표 계산 함수 정의
def calculate_indicators(df):
    # Parabolic SAR
    df['Parabolic_SAR_0.02'] = pata.psar(df['high'], df['low'], df['close'], af=0.02, max_af=0.2)['PSARl_0.02_0.2']
    df['Parabolic_SAR_0.04'] = pata.psar(df['high'], df['low'], df['close'], af=0.04, max_af=0.2)['PSARl_0.04_0.2']
    df['Parabolic_SAR_0.06'] = pata.psar(df['high'], df['low'], df['close'], af=0.06, max_af=0.2)['PSARl_0.06_0.2']
    df['Parabolic_SAR_0.08'] = pata.psar(df['high'], df['low'], df['close'], af=0.08, max_af=0.2)['PSARl_0.08_0.2']
    df['Parabolic_SAR_0.1'] = pata.psar(df['high'], df['low'], df['close'], af=0.1, max_af=0.2)['PSARl_0.1_0.2']

    # Williams %R
    df['Williams_%R_10'] = pata.willr(df['high'], df['low'], df['close'], length=10)
    df['Williams_%R_20'] = pata.willr(df['high'], df['low'], df['close'], length=20)
    df['Williams_%R_30'] = pata.willr(df['high'], df['low'], df['close'], length=30)
    df['Williams_%R_40'] = pata.willr(df['high'], df['low'], df['close'], length=40)
    df['Williams_%R_50'] = pata.willr(df['high'], df['low'], df['close'], length=50)

    # Momentum
    df['Momentum_10'] = pata.mom(df['close'], length=10)
    df['Momentum_20'] = pata.mom(df['close'], length=20)
    df['Momentum_30'] = pata.mom(df['close'], length=30)
    df['Momentum_40'] = pata.mom(df['close'], length=40)
    df['Momentum_50'] = pata.mom(df['close'], length=50)

    # Rate of Change (ROC)
    df['ROC_10'] = pata.roc(df['close'], length=10)
    df['ROC_20'] = pata.roc(df['close'], length=20)
    df['ROC_30'] = pata.roc(df['close'], length=30)
    df['ROC_40'] = pata.roc(df['close'], length=40)
    df['ROC_50'] = pata.roc(df['close'], length=50)

    # Chande Momentum Oscillator (CMO)
    df['CMO_10'] = pata.cmo(df['close'], length=10)
    df['CMO_20'] = pata.cmo(df['close'], length=20)
    df['CMO_30'] = pata.cmo(df['close'], length=30)
    df['CMO_40'] = pata.cmo(df['close'], length=40)
    df['CMO_50'] = pata.cmo(df['close'], length=50)

    # Money Flow Index (MFI)
    df['MFI_10'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=10)
    df['MFI_20'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=20)
    df['MFI_30'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=30)
    df['MFI_40'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=40)
    df['MFI_50'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=50)

    # Relative Strength Index (RSI)
    df['RSI_10'] = pata.rsi(df['close'], length=10)
    df['RSI_20'] = pata.rsi(df['close'], length=20)
    df['RSI_30'] = pata.rsi(df['close'], length=30)
    df['RSI_40'] = pata.rsi(df['close'], length=40)
    df['RSI_50'] = pata.rsi(df['close'], length=50)

    # Accumulation/Distribution Line (A/D Line)
    df['Accumulation_Distribution_Line'] = pata.ad(df['high'], df['low'], df['close'], df['volume'])

    # Elder's Force Index (EFI)
    df['Elder_Force_Index_2'] = pata.efi(df['close'], df['volume'], length=2)
    df['Elder_Force_Index_13'] = pata.efi(df['close'], df['volume'], length=13)
    df['Elder_Force_Index_5'] = pata.efi(df['close'], df['volume'], length=5)
    df['Elder_Force_Index_10'] = pata.efi(df['close'], df['volume'], length=10)
    df['Elder_Force_Index_25'] = pata.efi(df['close'], df['volume'], length=25)

    # Relative Vigor Index (RVI)
    df['Relative_Vigor_Index_10'] = pata.rsi(df['close'], length=10)  # pandas-ta는 RVI가 없으므로 RSI로 대체
    df['Relative_Vigor_Index_20'] = pata.rsi(df['close'], length=20)
    df['Relative_Vigor_Index_30'] = pata.rsi(df['close'], length=30)
    df['Relative_Vigor_Index_40'] = pata.rsi(df['close'], length=40)
    df['Relative_Vigor_Index_50'] = pata.rsi(df['close'], length=50)

    # Volume Ratio (VR)
    def volume_ratio(close, volume, period):
        vr = []
        for i in range(len(close)):
            if i < period:
                vr.append(None)
            else:
                vol_up = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] > close[j - 1])
                vol_down = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] < close[j - 1])
                vol_same = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] == close[j - 1])
                vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
                vr.append(vr_value)
        return vr

    df['VR_10'] = volume_ratio(df['close'], df['volume'], period=10)
    df['VR_20'] = volume_ratio(df['close'], df['volume'], period=20)
    df['VR_30'] = volume_ratio(df['close'], df['volume'], period=30)
    df['VR_40'] = volume_ratio(df['close'], df['volume'], period=40)
    df['VR_50'] = volume_ratio(df['close'], df['volume'], period=50)

    # Commodity Channel Index (CCI)
    df['CCI_10'] = pata.cci(df['high'], df['low'], df['close'], length=10)
    df['CCI_20'] = pata.cci(df['high'], df['low'], df['close'], length=20)
    df['CCI_30'] = pata.cci(df['high'], df['low'], df['close'], length=30)
    df['CCI_40'] = pata.cci(df['high'], df['low'], df['close'], length=40)
    df['CCI_50'] = pata.cci(df['high'], df['low'], df['close'], length=50)

    return df

# 지표 계산
data = calculate_indicators(data)

In [ ]:
# 이격도(Price Disparity Index) 계산 함수
def calculate_disparity_index(df, period):
    df[f'disparity_index_{period}'] = (df['close'] / df['close'].rolling(window=period).mean()) * 100
    return df

# 이평선(이동평균선) 계산 함수
def calculate_moving_averages(df, periods):
    for period in periods:
        df[f'price_ma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'volume_ma_{period}'] = df['volume'].rolling(window=period).mean()
    return df

# 이격도와 이동평균선 계산
disparity_periods = [5, 10, 20, 50, 100, 200]
moving_average_periods = [5, 10, 20, 50, 100, 200]

# 이격도 계산
for period in disparity_periods:
    data = calculate_disparity_index(data, period)

# 이동평균선 계산
data = calculate_moving_averages(data, moving_average_periods)

In [ ]:
def calculate_max_min_returns(df):
    window_size = 60

    # 'open_time' 열이 데이터프레임에 있는지 확인
    if 'open_time' not in df.columns:
        raise KeyError("'open_time' 열이 데이터프레임에 포함되어 있어야 합니다.")

    # 'open_time' 열을 datetime으로 변환
    df['open_time'] = pd.to_datetime(df['open_time'])

    # 인덱스 중복 확인 및 제거
    df = df.drop_duplicates(subset='open_time', keep='first').copy()

    # 인덱스를 'open_time'으로 설정
    df.set_index('open_time', inplace=True)

    # 현재 가격
    current_price = df['close']

    # 60분 윈도우를 적용하여 최대 및 최소 가격 계산
    rolling_max = df['high'].rolling(window=window_size, min_periods=1).max().shift(-window_size)
    rolling_min = df['low'].rolling(window=window_size, min_periods=1).min().shift(-window_size)

    # 현재 행의 close와 60분 윈도우 내의 최대 high와 최소 low를 비교하여 수익률 계산
    df['max_return_60min'] = ((rolling_max - current_price) / current_price) * 100
    df['min_return_60min'] = ((rolling_min - current_price) / current_price) * 100

    # 결측값을 적절히 처리 (예: 마지막 몇 행)
    df['max_return_60min'].fillna(0, inplace=True)
    df['min_return_60min'].fillna(0, inplace=True)

    # 인덱스 리셋
    df.reset_index(inplace=True)

    return df

# 최대 상승률과 최대 하락률 계산
data = calculate_max_min_returns(data)

data.head()

In [ ]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
data.head()

In [ ]:
len(data.columns)

In [ ]:
file_path3 = "/content/drive/MyDrive/Data/SOL_Data_30m_Indicator3.csv";
data.to_csv(file_path3, index=False);